In [5]:
import pandas as pd
import re
# from nltk.corpus import words
from nltk.tokenize import word_tokenize
import nltk
#nltk.download('punkt')
from textblob import Word
pd.options.display.width = 0

import spacy

pd.set_option('display.max_rows', 6000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
#pd.describe_option('display')
# import enchant

In [6]:
df = pd.read_csv('../../../../data_off/clarify/spanish_comorbidity/comorbidities_modified.csv', sep=',', error_bad_lines=True, encoding='utf-8')
df.shape

(5453, 4)

In [7]:
df['replacement'] = df['replacement'].str.replace(';', '')
df['replacement'] = df['replacement'].str.strip()

In [8]:
correction_dict = {'Benign BIRADS2 breast nodule': 'Bening breast nodule (BI-RADS)',
 'HT+ DA cavit': 'Hormone therapy',
 'Bilateral hydrosadenitis': 'Hidradenitis suppurativa',
 'SCASEST': 'Acute coronary syndrome',
 'IAM': 'Acute Myocardial Infarction',
 'Alcholic': 'Alcoholic',
 'Hypoacucia': 'Hearing loss',
 'Sd IAVD': 'Right ventricle Acute myocardial infarction',
 'Saphenous ischemia': 'Saphenous vein Ischemia'}

In [9]:
df.replace({'replacement':correction_dict}, inplace=True)

In [10]:
# Find Spanish Abbreviations

In [11]:
df_sp_google = pd.read_csv('../../../../data_off/clarify/spanish_comorbidity/comorbidities_spanish_english_google.csv', sep=',', error_bad_lines=True, encoding='utf-8')
print (df_sp_google.shape)

df_sp_deep = pd.read_csv('../../../../data_off/clarify/spanish_comorbidity/spanish_english_deepl.csv', sep=',', error_bad_lines=True, encoding='utf-8')
print (df_sp_deep.shape)

(5453, 1)
(5453, 1)


In [12]:
df['replacement_google'] = df_sp_google['value']
df['replacement_deepl'] = df_sp_deep['value']

In [13]:
df['replacement_google'] = df['replacement_google'].str.strip()
df['replacement_deepl'] = df['replacement_deepl'].str.strip()

In [14]:
def capitalize_(text):
    if type(text)==str:
        if len(text)>0:
            return text[0:1].capitalize()+text[1:]
    else:
        return text

df['replacement'] = df['replacement'].apply(capitalize_)
df['replacement_google'] = df['replacement_google'].apply(capitalize_)
df['replacement_deepl'] = df['replacement_deepl'].apply(capitalize_)

In [15]:
# Calculate Abbreviations
def find_in_english_dict(text):
    for w in word_tokenize(text):
        if Word(w.strip().lower()).spellcheck()[0][1]!=1.0 and len(re.findall(r"[A-Z]{2,}", w))>0:
          # print(w)
          return True
    return False

df['haveAbbr'] = (df['value'].apply(lambda x : len(re.findall(r"[A-Z]{2,}", x))>0)) & \
    (df['replacement_google'].apply(find_in_english_dict))

In [22]:
# Calculate no of stop words
nlp = spacy.load('en_core_web_sm')
all_stopwords = nlp.Defaults.stop_words

def stop_words_length(text):
    # print (word_tokenize(str(text)))
    return len([word for word in word_tokenize(str(text)) if word in all_stopwords] and not word.isdigit())

df['stop_wrods_google_len'] = df['replacement_google'].apply(stop_words_length)
df['stop_words_deepl_len'] = df['replacement_deepl'].apply(stop_words_length)

In [32]:
' '.join(['a','b','c'])

'a b c'

In [50]:
' '.join([word for word in word_tokenize(str('I am the one 1 and 2 in 2011 and 2011A and rest and peeock')) if not word.isdigit() and word not in all_stopwords])

'I 2011A rest'

In [ ]:
# Remove stop words and years
def remove_stopwords_numbers(text):
    return ' '.join([word for word in word_tokenize(str(text)) if word not in all_stopwords or not word.isdigit()])

df['replacement_google_withst_num'] = df['replacement_google'].apply(stop_words_length)
df['replacement_google_withst_num'] = df['replacement_deepl'].apply(stop_words_length)


In [23]:
df.to_csv('../../../../data_off/clarify/spanish_comorbidity/spanish_google_deep_last.csv', index=False, encoding='utf-8')

In [24]:
df[df['haveAbbr']==True].to_csv('../../../../data_off/clarify/spanish_comorbidity/spanish_google_deep_last_index.csv', index=True, encoding='utf-8')

In [25]:
df.head(5)

,table_name,column_name,value,replacement,replacement_google,replacement_deepl,haveAbbr,stop_wrods_google_len,stop_words_deepl_len
0,comorbidity,positive_negative,0,NaN,0,0,False,0,0
1,comorbidity,positive_negative,1,NaN,1,1,False,0,0
2,comorbidity_treatment_line,comorbidity,Gastritis crÛnica,Chronic gastritis,Chronic gastritis,Chronic gastritis,False,0,0
3,comorbidity_treatment_line,comorbidity,Sarcoma PleomÛrfico Pectoral Mayor,Pectoralis major pleomorphic sarcoma,Pectoralis Major Pleomorphic Sarcoma,Pectoralis major pectoralis pleomÛrphic sarcoma,False,0,0
4,comorbidity_treatment_line,comorbidity,hiperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,False,0,0


In [26]:
df.tail(5)

,table_name,column_name,value,replacement,replacement_google,replacement_deepl,haveAbbr,stop_wrods_google_len,stop_words_deepl_len
5448,comorbidity_treatment_line,comorbidity,AIT en 2017,None,AIT in 2017,AIT in 2017,True,1,1
5449,comorbidity_treatment_line,comorbidity,pericarditis en 2013,None,Pericarditis in 2013,Pericarditis in 2013,False,1,1
5450,comorbidity_treatment_line,comorbidity,hepatopatia crÛnica con fibrosis e HTP por VHB,None,Chronic liver disease with fibrosis and HBV PHT,Chronic hepatopathy with fibrosis and HBV-related PHT,True,2,2
5451,comorbidity_treatment_line,comorbidity,enfermedad de crhon,None,Crhon disease,Crhon's disease,False,0,1
5452,comorbidity_treatment_line,comorbidity,intervenido en 3 ocasiones por oclusiÛn intestinal ultima complicada y es portador de fistula entero,None,Operated on 3 occasions for complicated last intestinal occlusion and is a carrier of an entire fistula,Operated 3 times for last complicated intestinal occlusion and is a carrier of an entire fistula.,False,8,7


In [27]:
# Train_data
df_MEV = df.dropna(subset=['replacement'])
df_MEV.to_csv('../../../../data_off/clarify/spanish_comorbidity/train_MEV.csv', index=False, encoding='utf-8')

In [28]:
# df_MEV.reset_index().iloc[[60,67,132,160,163,444,446,462,467,478,507,511,573,675,677,721,834]]['replacement'].tolist()

In [29]:
df_MEV

,table_name,column_name,value,replacement,replacement_google,replacement_deepl,haveAbbr,stop_wrods_google_len,stop_words_deepl_len
2,comorbidity_treatment_line,comorbidity,Gastritis crÛnica,Chronic gastritis,Chronic gastritis,Chronic gastritis,False,0,0
3,comorbidity_treatment_line,comorbidity,Sarcoma PleomÛrfico Pectoral Mayor,Pectoralis major pleomorphic sarcoma,Pectoralis Major Pleomorphic Sarcoma,Pectoralis major pectoralis pleomÛrphic sarcoma,False,0,0
4,comorbidity_treatment_line,comorbidity,hiperuricemia,Hyperuricemia,Hyperuricemia,Hyperuricemia,False,0,0
5,comorbidity_treatment_line,comorbidity,intervenido pÛlipo colon,Colon polyp operated,Colon plipo intervened,Intervened pÛlipo colon,False,0,0
6,comorbidity_treatment_line,comorbidity,intervenido estenosis aorta,Aortic stenosis operated,Operated aortic stenosis,Intervened aortic stenosis,False,0,0
7,comorbidity_treatment_line,comorbidity,Disnea,Dyspnea,Dyspnoea,Dyspnea,False,0,0
8,comorbidity_treatment_line,comorbidity,Osteoporosis,Osteoporosis,Osteoporosis,Osteoporosis,False,0,0
9,comorbidity_treatment_line,comorbidity,Histerectomia con doble anexectomia,Hysterectomy double adnexectomy,Hysterectomy with double adnexectomy,Hysterectomy with double adnexectomy,False,1,1
10,comorbidity_treatment_line,comorbidity,hipotiroidismo subclÌnico,Subclinical hypothyroidism,Subclinical hypothyroidism,Subclinical hypothyroidism,False,0,0
11,comorbidity_treatment_line,comorbidity,carcinoma de prÛstata en 2011,Prostate carcinoma,Prostate carcinoma in 2011,PrÛstata carcinoma in 2011,False,1,1
